# pandasでカテゴリ変数をダミー変数に変換（get_dummies）

https://note.nkmk.me/python-pandas-get-dummies/

pandasでカテゴリ変数（カテゴリカルデータ、質的データ）をダミー変数に変換するには、pandas.get_dummies()関数を使う。

pandas.get_dummies — pandas 0.22.0 documentation

文字列でカテゴリー分けされた性別などのデータを、男を0, 女を1のように変換したり、多クラスの特徴量をone-hot表現に変換したりすることができる。機械学習の前処理として行うことが多い。

ここでは以下の内容について説明する。

* pandas.get_dummies()の基本的な使い方
* 最初のカテゴリーを除外: 引数drop_first
* 欠損値NaNもダミー化: 引数dummy_na
* pandas.DataFrameのダミー変数の列名を指定: 引数prefix, prefix_sep
* pandas.DataFrameの列を指定して数値・ブール列もダミー化: 引数columns
* 各カテゴリー（水準）を任意の数値化: map()メソッド
* pandas.get_dummies()で複数のデータを変換する場合の注意点  

例として以下のデータを使用する。説明のために列を追加している。

In [5]:
import pandas as pd
import numpy as np

df = pd.read_csv('sample_pandas_normal.csv', index_col=0)

df['sex'] = ['female', np.nan, 'male', 'male', 'female', 'male']
df['rank'] = [2, 1, 1, 0, 2, 0]

df

,age,state,point,sex,rank
name,,,,,
Alice,24,NY,64,female,2
Bob,42,CA,92,NaN,1
Charlie,18,CA,70,male,1
Dave,68,TX,70,male,0
Ellen,24,CA,88,female,2
Frank,30,NY,57,male,0


## pandas.get_dummies()の基本的な使い方

第一引数dataにpanas.Series、配列（Pythonのリスト、NumPy配列ndarrayなど）、および、pandas.DataFrameを指定する。

いずれの場合もpandas.DataFrameの新たなオブジェクトが返される。元のオブジェクトを更新したい場合は、例えば以下のように元のオブジェクトに代入すればよい。

In [7]:
df = pd.get_dummies(df)
df

,age,point,rank,state_CA,state_NY,state_TX,sex_female,sex_male
name,,,,,,,,
Alice,24,64,2,0,1,0,1,0
Bob,42,92,1,1,0,0,0,0
Charlie,18,70,1,1,0,0,0,1
Dave,68,70,0,0,0,1,0,1
Ellen,24,88,2,1,0,0,1,0
Frank,30,57,0,0,1,0,0,1


### 引数にpandas.Series、配列を指定する場合

pandas.Series、配列（Pythonのリスト、NumPy配列ndarrayなど）の場合、カテゴリー名が列名になる。

In [12]:
df = pd.read_csv('sample_pandas_normal.csv', index_col=0)

df['sex'] = ['female', np.nan, 'male', 'male', 'female', 'male']
df['rank'] = [2, 1, 1, 0, 2, 0]

df

,age,state,point,sex,rank
name,,,,,
Alice,24,NY,64,female,2
Bob,42,CA,92,NaN,1
Charlie,18,CA,70,male,1
Dave,68,TX,70,male,0
Ellen,24,CA,88,female,2
Frank,30,NY,57,male,0


In [13]:
print(pd.get_dummies(df['sex']))

         female  male
name                 
Alice         1     0
Bob           0     0
Charlie       0     1
Dave          0     1
Ellen         1     0
Frank         0     1


In [14]:
print(pd.get_dummies(['male', 1, 1, 2]))

   1  2  male
0  0  0     1
1  1  0     0
2  1  0     0
3  0  1     0


In [15]:
print(pd.get_dummies(np.arange(6)))

   0  1  2  3  4  5
0  1  0  0  0  0  0
1  0  1  0  0  0  0
2  0  0  1  0  0  0
3  0  0  0  1  0  0
4  0  0  0  0  1  0
5  0  0  0  0  0  1


### 引数にpandas.DataFrameを指定する場合

pandas.DataFrameの場合は、デフォルトではデータ型dtypeがobject（おもに文字列）またはcategoryである列がすべてダミー変数化される。

数値（int, float）やブールboolの列は変換されず元のまま。数値やブールの列もダミー化したい場合の設定については後述。

pandas.DataFrameの場合の列名は元の列名_カテゴリー名となる。変更する設定は後述。


In [16]:
print(pd.get_dummies(df, drop_first=True))

         age  point  rank  state_NY  state_TX  sex_male
name                                                   
Alice     24     64     2         1         0         0
Bob       42     92     1         0         0         0
Charlie   18     70     1         0         0         1
Dave      68     70     0         0         1         1
Ellen     24     88     2         0         0         0
Frank     30     57     0         1         0         1


### 欠損値NaNもダミー化: 引数dummy_na

デフォルトでは欠損値NaNは除外して処理される。

NaNも一つのカテゴリーとしてダミー変数化したい場合は、引数dummy_na=Trueとする。

このとき、NaNを含まない列に対してもNaNのダミー変数が生成される。すべての要素が0となる。

In [17]:
print(pd.get_dummies(df, drop_first=True, dummy_na=True))

         age  point  rank  state_NY  state_TX  state_nan  sex_male  sex_nan
name                                                                       
Alice     24     64     2         1         0          0         0        0
Bob       42     92     1         0         0          0         0        1
Charlie   18     70     1         0         0          0         1        0
Dave      68     70     0         0         1          0         1        0
Ellen     24     88     2         0         0          0         0        0
Frank     30     57     0         1         0          0         1        0
